In [ ]:
! pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

In [ ]:
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from transformers import AutoTokenizer
from trl import SFTTrainer
import os
import pandas as pd
# import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
from datasets import Dataset

In [ ]:
data = pd.read_csv('/content/image.csv')

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.drop(columns=['Unnamed: 4'],inplace=True,axis=1)

In [ ]:
data.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
# tokenized_data = []

# for index, item in data.iterrows():
#     print(item)
#     inputs = tokenizer.encode_plus(item['Medicine Name'], return_tensors='pt', padding=True, truncation=True)

#     labels = torch.tensor([item['Composition'], item['Uses'], item['Side_effects'], item['Manufacturer'], item['Excellent Review %'], item['Average Review %'], item['Poor Review %']])

#     tokenized_data.append((inputs, labels))


In [ ]:

def generate_text(medicine_name, uses, side_effects, composition,Manufacturer,Excellent_Review,Average_Review,Poor_Review):
    return f"###Human : {medicine_name} ###Assistant : Uses: {uses} Side Effects : {side_effects} Composition: {composition} Manufacturer : {Manufacturer} Excellent_Review: {Excellent_Review} Average_Review :{Average_Review} Poor_Review:{Poor_Review}"


data["text"] = data.apply(lambda row: generate_text(row["Medicine Name"], row["Uses"], row["Side_effects"], row["Composition"],row['Manufacturer'],row['Excellent Review %'],row['Average Review %'],row['Poor Review %']), axis=1)

print(data)


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                            "mistralai/Mixtral-8x7B-Instruct-v0.1",
                            quantization_config=bnb_config,
                            device_map="auto"
                        )


model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
train_dataset = Dataset.from_pandas(data)

In [ ]:
training_arguments = TrainingArguments(
        output_dir="Medicine_finetuned",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=5,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        push_to_hub=False
)

trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("Medicine_finetuned")
tokenizer.save_pretrained("Medicine_finetuned")